In [1]:
import os
# import plotly.graph_objects as go
from tqdm import tqdm
from gym import envs
import argparse
import numpy as np
from bigmdp.data.buffer import StandardBuffer,ReplayBuffer, gather_data_in_buffer, get_iter_indexes
from bigmdp.mdp.MDP_GPU import FullMDP
from bigmdp.utils.utils_eval import evaluate_on_env
from bigmdp.mdp.agent import SimpleAgent
from copy import deepcopy as cpy
from os import path
from arg_def import * 
import gym
from sklearn.neighbors import KDTree
from IPython import display
import torch
import time
import copy

In [2]:
parser, ArgumentDict = get_argument_parser()
options = "--env_name CartPole-v0 --MAX_S_COUNT 15000 --MAX_NS_COUNT 5 --mdp_build_k 5 --policy_k 11 --normalize_by_distance"
args = parser.parse_args(options.split(" "))

for title, arg_names in ArgumentDict.items():
    print_args(args, to_show_args=arg_names, title = title)

##################################################    envArgs    ##################################################
env_name                       : CartPole-v0                    seed                           : 4444
###################################################################################################################
##################################################    mdpBuildArgs    ##################################################
unknown_transition_reward      : -1000                          rmax_reward                    : 10000
balanced_exploration           : 0                              rmax_threshold                 : 2
MAX_S_COUNT                    : 15000                          def_device                     : GPU
fill_with                      : 0Q_src-KNN                     mdp_build_k                    : 5
knn_delta                      : 1e-08                          penalty_type                   : linear
penalty_beta                   : 1     

In [3]:
env = gym.make(args.env_name)

In [4]:
train_buffer = ReplayBuffer(state_dim = env.observation_space.shape[0],
                           is_atari= False, 
                           atari_preprocessing= None, 
                           batch_size=32, 
                           buffer_size=20000,
                           device="cpu")

In [5]:
class DummyNet():
    def __init__(self, sim, add_noise=False):
        self.simulator = sim

    def encode_single(self, o):
        return tuple(o)

    def encode_batch(self, o_batch):
        return [tuple(o) for o in o_batch]

    def predict_single_transition(self, o, a):
        assert False, "Not Implemented Error"

    def predict_batch_transition(self, o_batch, a_batch):
        assert False, "Not Implemented Error"

In [6]:
empty_MDP = FullMDP(A= list(range(env.action_space.n)),
                    ur=args.unknown_transition_reward,
                    vi_params={"gamma":  args.gamma,
                               "slip_prob": args.slip_probability,
                               "rmax_reward": args.rmax_reward,
                               "rmax_thres": args.rmax_threshold,
                               "balanced_explr": args.balanced_exploration,
                              "rmin":-1000},
                    knn_delta=args.knn_delta,
                    MAX_S_COUNT=args.MAX_S_COUNT,
                    MAX_NS_COUNT=args.MAX_NS_COUNT,
                    default_mode=args.def_device)

myAgent =  SimpleAgent(mdp_T= empty_MDP, net = DummyNet(None), fill_with = args.fill_with,
                       mdp_build_k = args.mdp_build_k, plcy_k = args.policy_k[0],  
                       kNN_on_sa = args.smooth_with_seen, soft_at_plcy = args.soft_q, 
                       normalize_by_distance= args.normalize_by_distance,
                       penalty_type=args.penalty_type, penalty_beta = args.penalty_beta,abstraction_flag=False)

# myAgent.build_mdp(train_buffer, verbose= True)

In [7]:
eval_rewards = []
for i in range(20):
    train_buffer, info = gather_data_in_buffer(train_buffer, env,policy = lambda s:np.random.randint(2), episode_count=5, frame_count=1000)
    myAgent.build_mdp(train_buffer, verbose= False)
    eval_reward = evaluate_on_env(env,myAgent.policies["optimal"], eps_count=50,progress_bar=True)[0]
    eval_rewards.append(eval_reward)
    print("#"*20 + f"\t Episode: {i*5}, Size of MDP:{np.sum(myAgent.mdp_T.filled_mask)}, Eval Reward:{eval_reward}\t" + "#"*20)
        


100%|██████████| 212/212 [00:00<00:00, 3336.03it/s]

Average Reward of collected trajectories:21.2
Len of to commit transitions 105
ABstraction Faldg False
Len of to commit sa pairs 212


Elapsed Time:0s, VI Error:0.0821991, #Backups: 250
Elapsed Time:1s, VI Error:0.00666046, #Backups: 500


 12%|█▏        | 6/50 [00:00<00:00, 57.50it/s]

Elapsed Time:2s, VI Error:0.00057983, #Backups: 750
Time takedn to solve 2.0695133209228516


...

####################	 Episode: 90, Size of MDP:2006, Eval Reward:200.0	####################
Average Reward of collected trajectories:42.4
Len of to commit transitions 2215
ABstraction Faldg False
Len of to commit sa pairs 4432


100%|██████████| 4432/4432 [00:01<00:00, 3679.86it/s]


Elapsed Time:0s, VI Error:0.00022125, #Backups: 6750
Time takedn to solve 0.6390211582183838


100%|██████████| 50/50 [00:03<00:00, 14.97it/s]

####################	 Episode: 95, Size of MDP:2218, Eval Reward:200.0	####################
